### Todo

- Add text, titles, sections etc.
- Add reference to paper and IRM image for context
- A bit more data exploration
- Handle categorical attribute
- Mention at the end: 
    - Feature selection (e.g. MRMR)
    - Model selection (e.g. pycaret)
    - Model finetuning (e.g. optuna)

In [57]:
# Installing the repository in Google Colab

# # Disable cell output
# %%capture

# # Download the repository from github
# !git clone https://github.com/5TuX/ml-radiomics-classification.git

# # Set working directory
# %cd ml-radiomics-classification

# # Install dependencies with uv
# !curl -LsSf https://astral.sh/uv/install.sh | sh
# import os
# os.environ["PATH"] = f"/root/.cargo/bin:{os.environ['PATH']}"
# !uv sync

In [58]:
# Imports

import polars as pl
from itertools import groupby
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

Notebook settings:

In [59]:
settings = {
    "dataset-path": "data/Radiomics_binWidth-15_ZScore_NETnNCR_T1CE.csv",
}

Load the data:

In [60]:
df = pl.read_csv(settings["dataset-path"])
print("Dataset shape:", df.shape)

Dataset shape: (369, 1720)


Look at a few examples:

In [61]:
df.head()

Patient_ID,Group,Group_label,binWidth,Normalization,Age,Survival_days,Extent_of_Resection,Subregion,Sequence,diagnostics_Versions_PyRadiomics,diagnostics_Versions_Numpy,diagnostics_Versions_SimpleITK,diagnostics_Versions_PyWavelet,diagnostics_Versions_Python,diagnostics_Configuration_Settings,diagnostics_Configuration_EnabledImageTypes,diagnostics_Image-original_Hash,diagnostics_Image-original_Dimensionality,diagnostics_Image-original_Spacing,diagnostics_Image-original_Size,diagnostics_Image-original_Mean,diagnostics_Image-original_Minimum,diagnostics_Image-original_Maximum,diagnostics_Mask-original_Hash,diagnostics_Mask-original_Spacing,diagnostics_Mask-original_Size,diagnostics_Mask-original_BoundingBox,diagnostics_Mask-original_VoxelNum,diagnostics_Mask-original_VolumeNum,diagnostics_Mask-original_CenterOfMassIndex,diagnostics_Mask-original_CenterOfMass,original_shape_Elongation,original_shape_Flatness,original_shape_LeastAxisLength,original_shape_MajorAxisLength,original_shape_Maximum2DDiameterColumn,…,wavelet-LLL_glrlm_GrayLevelNonUniformity,wavelet-LLL_glrlm_GrayLevelNonUniformityNormalized,wavelet-LLL_glrlm_GrayLevelVariance,wavelet-LLL_glrlm_HighGrayLevelRunEmphasis,wavelet-LLL_glrlm_LongRunEmphasis,wavelet-LLL_glrlm_LongRunHighGrayLevelEmphasis,wavelet-LLL_glrlm_LongRunLowGrayLevelEmphasis,wavelet-LLL_glrlm_LowGrayLevelRunEmphasis,wavelet-LLL_glrlm_RunEntropy,wavelet-LLL_glrlm_RunLengthNonUniformity,wavelet-LLL_glrlm_RunLengthNonUniformityNormalized,wavelet-LLL_glrlm_RunPercentage,wavelet-LLL_glrlm_RunVariance,wavelet-LLL_glrlm_ShortRunEmphasis,wavelet-LLL_glrlm_ShortRunHighGrayLevelEmphasis,wavelet-LLL_glrlm_ShortRunLowGrayLevelEmphasis,wavelet-LLL_glszm_GrayLevelNonUniformity,wavelet-LLL_glszm_GrayLevelNonUniformityNormalized,wavelet-LLL_glszm_GrayLevelVariance,wavelet-LLL_glszm_HighGrayLevelZoneEmphasis,wavelet-LLL_glszm_LargeAreaEmphasis,wavelet-LLL_glszm_LargeAreaHighGrayLevelEmphasis,wavelet-LLL_glszm_LargeAreaLowGrayLevelEmphasis,wavelet-LLL_glszm_LowGrayLevelZoneEmphasis,wavelet-LLL_glszm_SizeZoneNonUniformity,wavelet-LLL_glszm_SizeZoneNonUniformityNormalized,wavelet-LLL_glszm_SmallAreaEmphasis,wavelet-LLL_glszm_SmallAreaHighGrayLevelEmphasis,wavelet-LLL_glszm_SmallAreaLowGrayLevelEmphasis,wavelet-LLL_glszm_ZoneEntropy,wavelet-LLL_glszm_ZonePercentage,wavelet-LLL_glszm_ZoneVariance,wavelet-LLL_ngtdm_Busyness,wavelet-LLL_ngtdm_Coarseness,wavelet-LLL_ngtdm_Complexity,wavelet-LLL_ngtdm_Contrast,wavelet-LLL_ngtdm_Strength
str,str,i64,str,str,f64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,f64,f64,f64,str,str,str,str,f64,f64,str,str,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""BraTS20_Training_001""","""HGG""",1,"""binWidth-15""","""ZScore""",60.463,"""289""","""GTR""","""NETnNCR""","""T1CE""","""v3.0.1""","""1.20.3""","""2.1.1""","""1.1.1""","""3.9.7""","""{'minimumROIDimensions': 2, 'm…","""{'Original': {}, 'Exponential'…","""fcf31c6f56b4067eb28299303a0674…","""3D""","""(1.0, 1.0, 1.0)""","""(240, 240, 155)""",-6.4336e-16,-0.404772,11.492377,"""f3599f6c7ce9538e47d18beaef7292…","""(1.0, 1.0, 1.0)""","""(240, 240, 155)""","""(70, 102, 41, 50, 56, 37)""",15443.0,34.0,"""(89.23389237842387, 122.326749…","""(89.23389237842387, 122.326749…",0.731829,0.41979,21.139285,50.356792,50.358713,…,2837.692308,1.0,0.0,1.0,56.108317,56.108317,56.108317,1.0,3.683174,314.441224,0.108017,0.183753,23.691856,0.28482,0.28482,0.28482,34.0,1.0,0.0,1.0,6.3840e6,6.3840e6,6.3840e6,1.0,5.470588,0.1609,0.379267,0.379267,0.379267,3.133984,0.002202,6.1777e6,0.0,1e6,0.0,0.0,0.0
"""BraTS20_Training_002""","""HGG""",1,"""binWidth-15""","""ZScore""",52.263,"""616""","""GTR""","""NETnNCR""","""T1CE""","""v3.0.1""","""1.20.3""","""2.1.1""","""1.1.1""","""3.9.7""","""{'minimumROIDimensions': 2, 'm…","""{'Original': {}, 'Exponential'…","""58a926796869be2fcbfafa87f1e2d3…","""3D""","""(1.0, 1.0, 1.0)""","""(240, 

Look at basic statistics:

In [62]:
df.describe()

statistic,Patient_ID,Group,Group_label,binWidth,Normalization,Age,Survival_days,Extent_of_Resection,Subregion,Sequence,diagnostics_Versions_PyRadiomics,diagnostics_Versions_Numpy,diagnostics_Versions_SimpleITK,diagnostics_Versions_PyWavelet,diagnostics_Versions_Python,diagnostics_Configuration_Settings,diagnostics_Configuration_EnabledImageTypes,diagnostics_Image-original_Hash,diagnostics_Image-original_Dimensionality,diagnostics_Image-original_Spacing,diagnostics_Image-original_Size,diagnostics_Image-original_Mean,diagnostics_Image-original_Minimum,diagnostics_Image-original_Maximum,diagnostics_Mask-original_Hash,diagnostics_Mask-original_Spacing,diagnostics_Mask-original_Size,diagnostics_Mask-original_BoundingBox,diagnostics_Mask-original_VoxelNum,diagnostics_Mask-original_VolumeNum,diagnostics_Mask-original_CenterOfMassIndex,diagnostics_Mask-original_CenterOfMass,original_shape_Elongation,original_shape_Flatness,original_shape_LeastAxisLength,original_shape_MajorAxisLength,…,wavelet-LLL_glrlm_GrayLevelNonUniformity,wavelet-LLL_glrlm_GrayLevelNonUniformityNormalized,wavelet-LLL_glrlm_GrayLevelVariance,wavelet-LLL_glrlm_HighGrayLevelRunEmphasis,wavelet-LLL_glrlm_LongRunEmphasis,wavelet-LLL_glrlm_LongRunHighGrayLevelEmphasis,wavelet-LLL_glrlm_LongRunLowGrayLevelEmphasis,wavelet-LLL_glrlm_LowGrayLevelRunEmphasis,wavelet-LLL_glrlm_RunEntropy,wavelet-LLL_glrlm_RunLengthNonUniformity,wavelet-LLL_glrlm_RunLengthNonUniformityNormalized,wavelet-LLL_glrlm_RunPercentage,wavelet-LLL_glrlm_RunVariance,wavelet-LLL_glrlm_ShortRunEmphasis,wavelet-LLL_glrlm_ShortRunHighGrayLevelEmphasis,wavelet-LLL_glrlm_ShortRunLowGrayLevelEmphasis,wavelet-LLL_glszm_GrayLevelNonUniformity,wavelet-LLL_glszm_GrayLevelNonUniformityNormalized,wavelet-LLL_glszm_GrayLevelVariance,wavelet-LLL_glszm_HighGrayLevelZoneEmphasis,wavelet-LLL_glszm_LargeAreaEmphasis,wavelet-LLL_glszm_LargeAreaHighGrayLevelEmphasis,wavelet-LLL_glszm_LargeAreaLowGrayLevelEmphasis,wavelet-LLL_glszm_LowGrayLevelZoneEmphasis,wavelet-LLL_glszm_SizeZoneNonUniformity,wavelet-LLL_glszm_SizeZoneNonUniformityNormalized,wavelet-LLL_glszm_SmallAreaEmphasis,wavelet-LLL_glszm_SmallAreaHighGrayLevelEmphasis,wavelet-LLL_glszm_SmallAreaLowGrayLevelEmphasis,wavelet-LLL_glszm_ZoneEntropy,wavelet-LLL_glszm_ZonePercentage,wavelet-LLL_glszm_ZoneVariance,wavelet-LLL_ngtdm_Busyness,wavelet-LLL_ngtdm_Coarseness,wavelet-LLL_ngtdm_Complexity,wavelet-LLL_ngtdm_Contrast,wavelet-LLL_ngtdm_Strength
str,str,str,f64,str,str,f64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,f64,f64,f64,str,str,str,str,f64,f64,str,str,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""count""","""369""","""369""",369.0,"""369""","""369""",236.0,"""236""","""129""","""369""","""369""","""368""","""368""","""368""","""368""","""368""","""368""","""368""","""368""","""368""","""368""","""368""",368.0,368.0,368.0,"""368""","""368""","""368""","""368""",368.0,368.0,"""368""","""368""",368.0,368.0,368.0,368.0,…,368.0,368.0,368.0,368.0,368.0,368.0,368.0,368.0,368.0,368.0,368.0,368.0,368.0,368.0,368.0,368.0,368.0,368.0,368.0,368.0,368.0,368.0,368.0,368.0,368.0,368.0,368.0,368.0,368.0,368.0,368.0,368.0,368.0,368.0,368.0,368.0,368.0
"""null_count""","""0""","""0""",0.0,"""0""","""0""",133.0,"""133""","""240""","""0""","""0""","""1""","""1""","""1""","""1""","""1""","""1""","""1""","""1""","""1""","""1""","""1""",1.0,1.0,1.0,"""1""","""1""","""1""","""1""",1.0,1.0,"""1""","""1""",1.0,1.0,1.0,1.0,…,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
"""mean""",null,null,0.794038,null,null,61.223203,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,-1.3152e-17,-0.419098,10.345853,null,null,null,null,22179.336957,33.103261,null,null,0.723042,0.555079,24.485618,45.730192,…,2963.677357,0.994174,0.002913,1.3

Explore structure of column names:

In [63]:
print(f"{len(df.columns)} columns in total.\n")


def key(colname):
    return colname.split("_")[0]


for column_group, columns in groupby(df.columns, key):
    print(column_group)
    columns = list(columns)
    if len(columns) == 1:
        print(f"\t-> {columns}")
        continue
    columns = [colname[len(column_group) + 1 :] for colname in columns]
    for column_subgroup, subcolumns in groupby(columns, key):
        maxlen, suffix = 3, ""
        subcolumns = [colname[len(column_subgroup) + 1 :] for colname in subcolumns]
        if (lensubcols := len(subcolumns)) > maxlen:
            subcolumns = subcolumns[:maxlen]
            suffix = f"... ({lensubcols} total)"
        print(f"\t{column_subgroup} -> {subcolumns} {suffix}")

1720 columns in total.

Patient
	-> ['Patient_ID']
Group
	 -> [''] 
	label -> [''] 
binWidth
	-> ['binWidth']
Normalization
	-> ['Normalization']
Age
	-> ['Age']
Survival
	-> ['Survival_days']
Extent
	-> ['Extent_of_Resection']
Subregion
	-> ['Subregion']
Sequence
	-> ['Sequence']
diagnostics
	Versions -> ['PyRadiomics', 'Numpy', 'SimpleITK'] ... (5 total)
	Configuration -> ['Settings', 'EnabledImageTypes'] 
	Image-original -> ['Hash', 'Dimensionality', 'Spacing'] ... (7 total)
	Mask-original -> ['Hash', 'Spacing', 'Size'] ... (8 total)
original
	shape -> ['Elongation', 'Flatness', 'LeastAxisLength'] ... (14 total)
	firstorder -> ['10Percentile', '90Percentile', 'Energy'] ... (18 total)
	glcm -> ['Autocorrelation', 'ClusterProminence', 'ClusterShade'] ... (24 total)
	gldm -> ['DependenceEntropy', 'DependenceNonUniformity', 'DependenceNonUniformityNormalized'] ... (14 total)
	glrlm -> ['GrayLevelNonUniformity', 'GrayLevelNonUniformityNormalized', 'GrayLevelVariance'] ... (16 total)
	gls

Select columns:

In [64]:
# Exclude metadata columns

df = df.select(
    pl.all()
    .exclude(
        [
            "Patient_ID",
            "Group_label",
            "binWidth",
            "Normalization",
            "Subregion",
            "Sequence",
        ]
    )
    .exclude("^diagnostics_.*$")
)

# Select inputs and targets columns

# Target variables: ignore Survival_days (too many missing values)
targets = ["Group"]
# Clinical inputs: ignore Age and Extent_of_Resection (too many missing values):
inputs_clinical = []
# Radiomic inputs: keep only basic radiomic features and ignore the rest:
inputs_radiomics = [col for col in df.columns if col.startswith("original_")]
df = df.select(targets + inputs_clinical + inputs_radiomics).rename(
    {col: col[9:] for col in inputs_radiomics}
)

Handle missing values:

In [65]:
df = df.drop_nulls()
print("Dataset shape after dropping nulls:", df.shape)

Dataset shape after dropping nulls: (368, 108)


Look again at a few examples and basic statistics:

In [66]:
df.head()

Group,shape_Elongation,shape_Flatness,shape_LeastAxisLength,shape_MajorAxisLength,shape_Maximum2DDiameterColumn,shape_Maximum2DDiameterRow,shape_Maximum2DDiameterSlice,shape_Maximum3DDiameter,shape_MeshVolume,shape_MinorAxisLength,shape_Sphericity,shape_SurfaceArea,shape_SurfaceVolumeRatio,shape_VoxelVolume,firstorder_10Percentile,firstorder_90Percentile,firstorder_Energy,firstorder_Entropy,firstorder_InterquartileRange,firstorder_Kurtosis,firstorder_Maximum,firstorder_MeanAbsoluteDeviation,firstorder_Mean,firstorder_Median,firstorder_Minimum,firstorder_Range,firstorder_RobustMeanAbsoluteDeviation,firstorder_RootMeanSquared,firstorder_Skewness,firstorder_TotalEnergy,firstorder_Uniformity,firstorder_Variance,glcm_Autocorrelation,glcm_ClusterProminence,glcm_ClusterShade,glcm_ClusterTendency,…,glrlm_GrayLevelNonUniformity,glrlm_GrayLevelNonUniformityNormalized,glrlm_GrayLevelVariance,glrlm_HighGrayLevelRunEmphasis,glrlm_LongRunEmphasis,glrlm_LongRunHighGrayLevelEmphasis,glrlm_LongRunLowGrayLevelEmphasis,glrlm_LowGrayLevelRunEmphasis,glrlm_RunEntropy,glrlm_RunLengthNonUniformity,glrlm_RunLengthNonUniformityNormalized,glrlm_RunPercentage,glrlm_RunVariance,glrlm_ShortRunEmphasis,glrlm_ShortRunHighGrayLevelEmphasis,glrlm_ShortRunLowGrayLevelEmphasis,glszm_GrayLevelNonUniformity,glszm_GrayLevelNonUniformityNormalized,glszm_GrayLevelVariance,glszm_HighGrayLevelZoneEmphasis,glszm_LargeAreaEmphasis,glszm_LargeAreaHighGrayLevelEmphasis,glszm_LargeAreaLowGrayLevelEmphasis,glszm_LowGrayLevelZoneEmphasis,glszm_SizeZoneNonUniformity,glszm_SizeZoneNonUniformityNormalized,glszm_SmallAreaEmphasis,glszm_SmallAreaHighGrayLevelEmphasis,glszm_SmallAreaLowGrayLevelEmphasis,glszm_ZoneEntropy,glszm_ZonePercentage,glszm_ZoneVariance,ngtdm_Busyness,ngtdm_Coarseness,ngtdm_Complexity,ngtdm_Contrast,ngtdm_Strength
str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""HGG""",0.731829,0.41979,21.139285,50.356792,50.358713,59.539903,50.990195,63.85922,15226.333333,36.852581,0.333812,8899.775397,0.584499,15443.0,0.891341,2.406696,43805.647206,0.000994,0.76735,3.905945,4.857057,0.476079,1.57019,1.471689,-0.037217,4.894274,0.322622,1.684222,0.845454,43805.647206,0.99987,0.371107,3.999682,0.000159,-0.000159,0.000159,…,2837.693033,0.999275,0.000362,3.998912,56.052892,224.210478,14.013495,0.250272,3.685716,315.192629,0.108203,0.183882,23.686554,0.285304,1.140127,0.071598,33.057143,0.94449,0.027755,3.914286,6.2008e6,2.4803e7,1.5502e6,0.271429,5.971429,0.170612,0.397002,1.502295,0.120679,3.231618,0.002266,6.0061e6,0.300826,0.831125,0.000156,9.2377e-9,0.90777
"""HGG""",0.805201,0.596898,17.919111,30.020387,34.132096,34.525353,31.144823,35.496479,9073.541667,24.172434,0.73062,2879.455924,0.317346,9160.0,0.511066,1.20927,7040.180407,-3.2034e-16,0.263257,10.595436,3.429786,0.233776,0.801736,0.711371,0.013167,3.416619,0.120232,0.876686,2.380337,7040.180407,1.0,0.125799,1.0,0.0,0.0,0.0,…,931.230769,1.0,0.0,1.0,148.738015,148.738015,148.738015,1.0,4.063811,65.188726,0.068125,0.101663,37.847387,0.148596,0.148596,0.148596,9.0,1.0,0.0,1.0,9.1449e6,9.1449e6,9.1449e6,1.0,1.0,0.111111,0.155282,0.155282,0.155282,3.169925,0.000983,8.1090e6,0.0,1e6,0.0,0.0,0.0
"""HGG""",0.768372,0.722447,11.550058,15.987407,18.681542,18.384776,18.681542,22.135944,684.083333,12.284284,0.406249,924.196735,1.351,733.0,1.441948,3.355092,4630.563368,-3.2034e-16,1.090258,2.45011,4.360985,0.592579,2.407835,2.420584,0.713811,3.647174,0.43292,2.513419,0.055125,4630.563368,1.0,0.519605,1.0,0.0,0.0,0.0,…,291.153846,1.0,0.0,1.0,10.77577,10.77577,10.77577,1.0,2.387787,77.714144,0.259495,0.397209,3.934461,0.500213,0.500213,0.500213,7.0,1.0,0.0,1.0,62367.571429,62367.571429,62367.571429,1.0,1.285714,0.183673,0.298127,0.298127,0.298127,2.521641,0.00955,51402.489796,0.0,1e6,

In [67]:
df.describe()

statistic,Group,shape_Elongation,shape_Flatness,shape_LeastAxisLength,shape_MajorAxisLength,shape_Maximum2DDiameterColumn,shape_Maximum2DDiameterRow,shape_Maximum2DDiameterSlice,shape_Maximum3DDiameter,shape_MeshVolume,shape_MinorAxisLength,shape_Sphericity,shape_SurfaceArea,shape_SurfaceVolumeRatio,shape_VoxelVolume,firstorder_10Percentile,firstorder_90Percentile,firstorder_Energy,firstorder_Entropy,firstorder_InterquartileRange,firstorder_Kurtosis,firstorder_Maximum,firstorder_MeanAbsoluteDeviation,firstorder_Mean,firstorder_Median,firstorder_Minimum,firstorder_Range,firstorder_RobustMeanAbsoluteDeviation,firstorder_RootMeanSquared,firstorder_Skewness,firstorder_TotalEnergy,firstorder_Uniformity,firstorder_Variance,glcm_Autocorrelation,glcm_ClusterProminence,glcm_ClusterShade,…,glrlm_GrayLevelNonUniformity,glrlm_GrayLevelNonUniformityNormalized,glrlm_GrayLevelVariance,glrlm_HighGrayLevelRunEmphasis,glrlm_LongRunEmphasis,glrlm_LongRunHighGrayLevelEmphasis,glrlm_LongRunLowGrayLevelEmphasis,glrlm_LowGrayLevelRunEmphasis,glrlm_RunEntropy,glrlm_RunLengthNonUniformity,glrlm_RunLengthNonUniformityNormalized,glrlm_RunPercentage,glrlm_RunVariance,glrlm_ShortRunEmphasis,glrlm_ShortRunHighGrayLevelEmphasis,glrlm_ShortRunLowGrayLevelEmphasis,glszm_GrayLevelNonUniformity,glszm_GrayLevelNonUniformityNormalized,glszm_GrayLevelVariance,glszm_HighGrayLevelZoneEmphasis,glszm_LargeAreaEmphasis,glszm_LargeAreaHighGrayLevelEmphasis,glszm_LargeAreaLowGrayLevelEmphasis,glszm_LowGrayLevelZoneEmphasis,glszm_SizeZoneNonUniformity,glszm_SizeZoneNonUniformityNormalized,glszm_SmallAreaEmphasis,glszm_SmallAreaHighGrayLevelEmphasis,glszm_SmallAreaLowGrayLevelEmphasis,glszm_ZoneEntropy,glszm_ZonePercentage,glszm_ZoneVariance,ngtdm_Busyness,ngtdm_Coarseness,ngtdm_Complexity,ngtdm_Contrast,ngtdm_Strength
str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""count""","""368""",368.0,368.0,368.0,368.0,368.0,368.0,368.0,368.0,368.0,368.0,368.0,368.0,368.0,368.0,368.0,368.0,368.0,368.0,368.0,368.0,368.0,368.0,368.0,368.0,368.0,368.0,368.0,368.0,368.0,368.0,368.0,368.0,368.0,368.0,368.0,…,368.0,368.0,368.0,368.0,368.0,368.0,368.0,368.0,368.0,368.0,368.0,368.0,368.0,368.0,368.0,368.0,368.0,368.0,368.0,368.0,368.0,368.0,368.0,368.0,368.0,368.0,368.0,368.0,368.0,368.0,368.0,368.0,368.0,368.0,368.0,368.0,368.0
"""null_count""","""0""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,…,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"""mean""",null,0.723042,0.555079,24.485618,45.730192,45.057,51.053797,49.333276,56.808197,21862.7851,32.004908,0.376242,9237.61258,0.923609,22179.336957,1.374584,2.522678,74696.113733,0.007873,0.626598,3.996735,4.162478,0.36554,1.920441,1.882498,0.571002,3.591476,0.261605,1.985013,0.372539,74696.113733,0.996985,0.23679,1.530893,0.006383,-0.003187,…,2971.81061,0.994031,0.002985,1.526069,100.816023,186.329359,79.43769,0.868483,3.329075,428.802215,0.183117,0.2571,38.653377,0.369848,0.53148,0.32944,34.180723,0.959979,0.02001,1.353266,3.3695e8,5.0550e8,2.9481e8,0.911684,9.541138,0.32483,0.43707,0.604474,0.395219,2.36478,0.008932,1.1143e8,14.646119,826087.014728,0.0014,0.000218,0.055786
"""std""",null,0.155188,0.147826,10.167512,19.301596,16.080009,20.17874,19.738679,21.27765,29860.458179,12.507605,0.155833,8357.631322,0.910963,29878.14703,0.524901,0.51373,101634.496284,0.067677,0.276941,3.48061,1.281748,0.143954,0.490192,0.526195,0.592212,1.55996,0.111509,0.474185,0.726591,101634.496284,0.032312,0.185438,1.140018,0.065065,0.030067,…,2678.237799,0.03868,0.01934,1.1305,143.823868,380.737352,123.451225,0.282625,1.007

Train/test split:

In [68]:
print("Split the data in Train and Test subsets.")

training_data, test_data = train_test_split(
    df,
    test_size=0.20,
    stratify=df["Group"],
    random_state=42,
)


def check_class_proportions(df: pl.DataFrame):
    summary = (
        df.group_by("Group")
        .agg(pl.len().alias("count"))
        .with_columns((pl.col("count") / pl.col("count").sum()).alias("proportion"))
        .sort("count", descending=True)
    )
    return summary


print("\nCheck class proportions in training dataset:")
print(check_class_proportions(training_data))

print("\nCheck class proportions in test dataset:")
print(check_class_proportions(test_data))

Split the data in Train and Test subsets.

Check class proportions in training dataset:
shape: (2, 3)
┌───────┬───────┬────────────┐
│ Group ┆ count ┆ proportion │
│ ---   ┆ ---   ┆ ---        │
│ str   ┆ u32   ┆ f64        │
╞═══════╪═══════╪════════════╡
│ HGG   ┆ 233   ┆ 0.792517   │
│ LGG   ┆ 61    ┆ 0.207483   │
└───────┴───────┴────────────┘

Check class proportions in test dataset:
shape: (2, 3)
┌───────┬───────┬────────────┐
│ Group ┆ count ┆ proportion │
│ ---   ┆ ---   ┆ ---        │
│ str   ┆ u32   ┆ f64        │
╞═══════╪═══════╪════════════╡
│ HGG   ┆ 59    ┆ 0.797297   │
│ LGG   ┆ 15    ┆ 0.202703   │
└───────┴───────┴────────────┘


Feature scaling (use training data only):

In [ ]:
std_scaler = StandardScaler()

training_data_numerical = training_data.select(pl.col(pl.Float64))
training_data_numerical_scaled = std_scaler.fit_transform(training_data_numerical)
training_data_numerical_scaled = pl.DataFrame(
    training_data_numerical_scaled, schema=training_data_numerical.columns
)

max_features_to_plot = 3
for feature in training_data_numerical_scaled.columns[:max_features_to_plot]:
    training_data_numerical_scaled[feature].plot.hist().show()

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)